In [1]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.styles import Font, Color, colors, PatternFill
import time, sys
import os
import glob
from copy import copy
from datetime import date

In [3]:
try:
    connection = mysql.connector.connect(host='localhost',
                                            user='root',
                                            password='acidbass')
    
    proyecto = 'proyecto1'

    mysql_Create_DataBase_Query = (f"CREATE DATABASE {proyecto};")

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print('Connected to MySQL Server version ', db_Info)

        cursor = connection.cursor()
        cursor.execute(mysql_Create_DataBase_Query)
        cursor.execute(f'USE {proyecto};')
        print("You're connected to database: ", proyecto)

except mysql.connector.Error as e:
    print('Error while connecting to MySQL', e)

try:
        mysql_Create_Table_Query = """CREATE TABLE matriz (unidad VARCHAR(255), num VARCHAR(255),
                                    nivel VARCHAR(255), dni VARCHAR(255), nombre VARCHAR(255),
                                    terreno FLOAT, ocupada FLOAT, techada FLOAT, comunes FLOAT,
                                    moneda VARCHAR(255), valor FLOAT, terrenousd FLOAT,
                                    edifusd FLOAT, comercialusd FLOAT, realizausd FLOAT,
                                    asegurausd FLOAT, tipocambio FLOAT,fecha DATE);"""

        cursor.execute(mysql_Create_Table_Query)
        print('matriz Table created successfully ')

except mysql.connector.Error as e:
    print('Failed to create table in MySQL: {}'.format(Error), e)

finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print('MySQL connection is closed')

Connected to MySQL Server version  5.7.28-log
Error while connecting to MySQL 1007 (HY000): Can&#39;t create database &#39;proyecto1&#39;; database exists
Failed to create table in MySQL: &lt;class &#39;mysql.connector.errors.Error&#39;&gt; 1046 (3D000): No database selected
MySQL connection is closed


In [8]:
# Recopilación data de Matriz
DataInMatrix = pd.read_excel('Formatos\DATA.xlsx', sheet_name='MATRIZ')
DataInMatrix.fillna(value = 0, inplace = True)
#DataInMatrix

In [9]:
unidad = DataInMatrix.iloc[:]['Unidad Inmobiliaria']
num = DataInMatrix.iloc[:]['No']
nivel = DataInMatrix.iloc[:]['Nivel ']
aOcupada = DataInMatrix.iloc[:]['Área ocupada (m²)']
aTechada = DataInMatrix.iloc[:]['Área techada (m²)']
moneda = DataInMatrix.iloc[:]['Moneda']
vVenta = DataInMatrix.iloc[:]['Valor de Venta']
vista = DataInMatrix.iloc[:]['VISTA']
#unidad, num, nivel, aOcupada, aTechada, moneda, vVenta, vista

In [11]:
# Recopilación data de cálculo
DataInGeneral = pd.read_excel('Formatos\DATA.xlsx', sheet_name='DATA')
# DataInGeneral

In [12]:
vut = float(DataInGeneral.iloc[:1]['VUT (USD)'])
vrc = float(DataInGeneral.iloc[:1]['VRC (USD)'])
tc = float(DataInGeneral.iloc[:1]['TC'])
# vut, vrc, tc

In [14]:
wbProyecto = openpyxl.load_workbook('Formatos\DATA PROYECTO.xlsx', read_only=True)
sheetProyecto = wbProyecto['Memoria']
eFinan = sheetProyecto['D7'].value
wbProyecto.close

<bound method Workbook.close of <openpyxl.workbook.workbook.Workbook object at 0x000001EBB64D75B0>>

In [15]:
vue = DataInGeneral[['VUE (USD)', 'Tipo', 'Descripción']]
vue.rename(index = DataInGeneral.iloc[:]['Clase'], inplace=True)
# vue

In [10]:
try:
    connection = mysql.connector.connect(host='localhost',
                                            user='root',
                                            password='acidbass')

    proyecto = 'proyecto1'

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print('Connected to MySQL Server version ', db_Info)

        cursor = connection.cursor()
        cursor.execute(f'USE {proyecto};')
        print("You're connected to database: ", proyecto)

except mysql.connector.Error as e:
    print('Error while connecting to MySQL', e)

Connected to MySQL Server version  5.7.28-log
You&#39;re connected to database:  proyecto1


In [11]:
for x in range(0, len(DataInMatrix), 1):

    try:

        mysql_Fill_Table_Query = ("""INSERT INTO matriz (unidad, num, nivel, dni, nombre,
                                    terreno, ocupada, techada, comunes, moneda, valor, 
                                    terrenousd, edifusd, comercialusd, realizausd,
                                    asegurausd, tipocambio, fecha)"""
                                    """VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
                                    %s, %s, %s, %s, %s, %s, %s);"""
                                    )

        aTerreno = round(float(aOcupada[x]) / 20 * 1.4, 2)
        aComunes = round(float(aTechada[x]) * 0.15, 2)
        vTerrenoUsd = round(aTerreno * vut / 100, 0) * 100

        if unidad[x] == 'Departamento':
            vComercialUsd = round( 
                                    float(vue.loc[vista[x]]['VUE (USD)']) * (float(aTechada[x]) +
                                    ((float(aOcupada[x]) - float(aTechada[x])) * 0.3)) / 100,
                                    0
                                ) * 100
        else:
            vComercialUsd = vue.loc[vista[x]]['VUE (USD)']

        vEdificaUsd = vComercialUsd - vTerrenoUsd
        vRealizaUsd = round(vComercialUsd * 0.8 / 100, 0) *100

        if eFinan == 'BANCO INTERNACIONAL DEL PERÚ S.A.A. - INTERBANK':
            iAseguraUsd = vComercialUsd
        else:
            iAseguraUsd = round(
                                float(aTechada[x]) * vrc * 1.15 / 100,
                                0
                            ) * 100
        
        datos = (unidad[x], str(num[x]), str(nivel[x]), None, None, float(aTerreno),
                float(aOcupada[x]), float(aTechada[x]), float(aComunes), moneda[x], 
                float(vVenta[x]), float(vTerrenoUsd), float(vEdificaUsd), float(vComercialUsd),
                float(vRealizaUsd), float(iAseguraUsd), float(tc), None)

        cursor.execute(mysql_Fill_Table_Query, datos)
        connection.commit() # Important so the query us executed
        print(f'matriz Table updated with {unidad[x]} {num[x]},  successfully ')

    except mysql.connector.Error as e:
        print('Failed to update table in MySQL: {}'.format(Error), e)
        connection.rollback()


matriz Table updated with Departamento 101,  successfully 
matriz Table updated with Departamento 102,  successfully 
matriz Table updated with Departamento 103,  successfully 
matriz Table updated with Departamento 104,  successfully 
matriz Table updated with Departamento 105,  successfully 
matriz Table updated with Departamento 106,  successfully 
matriz Table updated with Departamento 107,  successfully 
matriz Table updated with Departamento 108,  successfully 
matriz Table updated with Departamento 201,  successfully 
matriz Table updated with Departamento 202,  successfully 
matriz Table updated with Departamento 203,  successfully 
matriz Table updated with Departamento 204,  successfully 
matriz Table updated with Departamento 205,  successfully 
matriz Table updated with Departamento 206,  successfully 
matriz Table updated with Departamento 207,  successfully 
matriz Table updated with Departamento 208,  successfully 
matriz Table updated with Departamento 301,  successfull

In [12]:
if (connection.is_connected()):
    cursor.close()
    connection.close()
    print('MySQL connection is closed')

MySQL connection is closed


In [13]:
try:
    connection = mysql.connector.connect(host='localhost',
                                            user='root',
                                            password='acidbass')

    proyecto = 'proyecto1'

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print('Connected to MySQL Server version ', db_Info)

        cursor = connection.cursor()
        cursor.execute(f'USE {proyecto};')
        print("You're connected to database: ", proyecto)

        DataOutMatrix = pd.read_sql(F'SELECT * FROM matriz', connection)

except mysql.connector.Error as e:
    print('Error while connecting to MySQL', e)

finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print('MySQL connection is closed')

# DataOutMatrix

Connected to MySQL Server version  5.7.28-log
You&#39;re connected to database:  proyecto1
MySQL connection is closed


In [14]:
unidad = DataOutMatrix.iloc[:]['unidad']
num = DataOutMatrix.iloc[:]['num']
nivel = DataOutMatrix.iloc[:]['nivel']
dni = DataOutMatrix.iloc[:]['dni']
nombre = DataOutMatrix.iloc[:]['nombre']
aTerreno = DataOutMatrix.iloc[:]['terreno']
aOcupada = DataOutMatrix.iloc[:]['ocupada']
aTechada = DataOutMatrix.iloc[:]['techada']
aComunes = DataOutMatrix.iloc[:]['comunes']
moneda = DataOutMatrix.iloc[:]['moneda']
vVenta = DataOutMatrix.iloc[:]['valor']
vTerrenoUsd = DataOutMatrix.iloc[:]['terrenousd']
vEdificaUsd = DataOutMatrix.iloc[:]['edifusd']
vComercialUsd = DataOutMatrix.iloc[:]['comercialusd']
vRealizaUsd = DataOutMatrix.iloc[:]['realizausd']
vAseguraUsd = DataOutMatrix.iloc[:]['asegurausd']
tc = DataOutMatrix.iloc[:]['tipocambio']
fecha = DataOutMatrix.iloc[:]['fecha']
# unidad, num, nivel, dni, nombre, aTerreno,aOcupada, aTechada, aComunes, moneda, vVenta, vTerrenoUsd, vEdificaUsd, vComercialUsd, vRealizaUsd, vAseguraUsd, tc, fecha

In [15]:
wbProyecto = openpyxl.load_workbook('Formatos\DATA PROYECTO.xlsx', read_only=True)
sheetProyecto = wbProyecto['Portada']
wbMatrix = openpyxl.load_workbook('Formatos\MATRIZ TASACIONES.xlsx', read_only=False)
sheetMatrix = wbMatrix['MATRIZ']

for x in range(0, len(DataOutMatrix), 1):
    vTerrenoSol = round(float(vTerrenoUsd[x]) * float(tc[x]) / 100, 0) * 100
    vEdificaSol = round(float(vEdificaUsd[x]) * float(tc[x]) / 100, 0) * 100
    vComercialSol = round(float(vComercialUsd[x]) * float(tc[x]) / 100, 0) * 100
    vRealizaSol = round(float(vRealizaUsd[x]) * float(tc[x]) / 100, 0) * 100
    vAseguraSol = round(float(vAseguraUsd[x]) * float(tc[x]) / 100, 0) * 100

    rowData = [unidad[x], num[x], nivel[x], dni[x], nombre[x], aTerreno[x], aOcupada[x],
                aTechada[x], aComunes[x], moneda[x], vVenta[x], aTerreno[x], aOcupada[x], 
                aTechada[x], aComunes[x], vTerrenoUsd[x], vEdificaUsd[x], vComercialUsd[x],
                vRealizaUsd[x], vAseguraUsd[x], tc[x], vTerrenoSol, vEdificaSol, vComercialSol,
                vRealizaSol, vAseguraSol, fecha[x]]

    sheetMatrix.insert_rows(x+13)

    for y in range(0, len(rowData), 1):
        sheetMatrix.cell(row=x+12, column=y+1).value = rowData[y]
        
        if sheetMatrix.cell(row=x+12, column=y+1).has_style:
            sheetMatrix.cell(row=x+13, column=y+1)._style = sheetMatrix.cell(row=x+12, column=y+1)._style
        
        if not fecha[x] == None:
            sheetMatrix.cell(row=x+12, column=y+1).font = Font(bold=True)
            sheetMatrix.cell(row=x+12, column=y+1).fill = PatternFill("solid", fgColor="DDEBF7")

sheetMatrix[f'F{x+14}'] = f'=SUM(F12:F{x+13})'
sheetMatrix[f'G{x+14}'] = f'=SUM(G12:G{x+13})'
sheetMatrix[f'H{x+14}'] = f'=SUM(H12:H{x+13})'
sheetMatrix[f'I{x+14}'] = f'=SUM(I12:I{x+13})'
sheetMatrix['D7'] = f'=MAX(AA12:AA{x+13})'
sheetMatrix['D8'] = time.strftime("%d/%m/%Y", time.gmtime(time.time()))


sheetMatrix['D3'].value = sheetProyecto['B34'].value
sheetMatrix['D5'].value = sheetProyecto['B33'].value

sheetProyecto = wbProyecto['Memoria']
sheetMatrix['D6'].value = sheetProyecto['D31'].value

wbMatrix.save('C:\\Users\\pollo\\Desktop\\Matriz.xlsx')
wbMatrix.close
wbProyecto.close

&lt;bound method Workbook.close of &lt;openpyxl.workbook.workbook.Workbook object at 0x000002A045CD7F70&gt;&gt;

In [2]:
#CREAR INFORME DE TASACIÓN

In [2]:
try:
    connection = mysql.connector.connect(host='localhost',
                                            user='root',
                                            password='acidbass')

    proyecto = 'proyecto1'

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print('Connected to MySQL Server version ', db_Info)

        cursor = connection.cursor()
        cursor.execute(f'USE {proyecto};')
        print("You're connected to database: ", proyecto)

        DataOutMatrix = pd.read_sql(F'SELECT * FROM matriz', connection)

except mysql.connector.Error as e:
    print('Error while connecting to MySQL', e)

finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print('MySQL connection is closed')

Connected to MySQL Server version  5.7.28-log
You're connected to database:  proyecto1
MySQL connection is closed


In [3]:
#Según selección toma datos de matriz y muestra en pantalla de usuario con opción de edición
# selected for test = 1, 183, 224
key = {0:1, 1:183, 2:224}
tasacionNew = pd.DataFrame(columns= DataOutMatrix.columns)
for x in key:
    tasacionNew = tasacionNew.append(DataOutMatrix.loc[key[x]])
tasacionNew

,unidad,num,nivel,dni,nombre,terreno,ocupada,techada,comunes,moneda,...,edifusd,comercialusd,realizausd,asegurausd,tipocambio,fecha,clase,vueusd,tipo,descripción
1,Departamento,102,1.0,None,None,4.38,62.50,62.50,9.38,Soles,...,65700.0,68800.0,55000.0,68800.0,3.5,None,Dpto 1150,1100.0,Tríplex,"Ingreso, sala comedor, cocina, lavanderia, hal..."
183,Estacionamiento,1047,0.0,None,None,0.84,12.00,12.00,1.80,Soles,...,8400.0,9000.0,7200.0,9000.0,3.5,None,Estac 28k,9000.0,Techado,Estacionamiento
224,Depósito,11,-3.0,None,None,0.44,6.24,6.24,0.94,Soles,...,3700.0,4000.0,3200.0,4000.0,3.5,None,Dep,4000.0,-,Depósito


In [4]:
unidad = tasacionNew.iloc[:]['unidad']
num = tasacionNew.iloc[:]['num']
nivel = tasacionNew.iloc[:]['nivel']
aTerreno = tasacionNew['terreno']
aTechada = tasacionNew['techada']
aOcupada = tasacionNew['ocupada']
descrip = tasacionNew['descripción']
vComercialUsd = tasacionNew['comercialusd']

In [5]:
#Selección por parte del Usuario
codigoTasa = '256'
cliente = 'Pepito Mendienta - DNI: 45768939'
fechaInspecc = '20/10/2020'
tcTasa = float('3.6')
# unidadSelected[0]

In [6]:
wbProyecto = openpyxl.load_workbook('Formatos\PMF-Tasación.xlsx', read_only=True)
sheetInforme = wbProyecto['Memoria']
banco = sheetInforme['D7'].value
wbProyecto.close

<bound method Workbook.close of <openpyxl.workbook.workbook.Workbook object at 0x00000206CA7395B0>>

In [7]:
if banco == 'BANCO INTERNACIONAL DEL PERÚ S.A.A. - INTERBANK':
    banco = 'IBK'
elif banco == 'BANCO PINCHINCHA':
    banco = 'BP'
elif banco == 'SCOTIABANK PERÚ S.A.A.':
    banco = 'SCB'

In [8]:
codigoTasa = f'Informe de Valuación N° {codigoTasa}-{banco}-{str(date.today().strftime("%Y"))}'
codigoTasa

'Informe de Valuación N° 256-IBK-2020'

In [9]:
tipo = ''
for x in range(0, len(key), 1):
    if x == 0:
        tipo = unidad[key[x]]
    else:
        if not unidad[key[x]] == unidad[key[x-1]]:
            tipo = f"{tipo}, {unidad[key[x]]}"
tipo

'Departamento, Estacionamiento, Depósito'

In [10]:
DataInGeneral = pd.read_excel('Formatos\DATA.xlsx', sheet_name='DATA')

vut = float(DataInGeneral.iloc[:1]['VUT (USD)']) #Unit terrain value
vrc = float(DataInGeneral.iloc[:1]['VRC (USD)']) #Unit reconstruction value

In [35]:
wbInforme = openpyxl.load_workbook('Formatos\Informe.xlsx', read_only=False)
sheetInforme = wbInforme['Informe']

In [30]:
filasInforme = [376, 350, 334, 316, 297, 210, 199, 180, 164, 148, 110, 65, 51]

In [31]:
sheetInforme['B4'].value = codigoTasa
sheetInforme['I7'].value = cliente
sheetInforme['I12'].value = fechaInspecc
sheetInforme['E67'].value = tcTasa
sheetInforme['J74'].value = tipo

In [32]:
y = 0
for x in range(0, len(key), 1):
    sheetInforme.cell(row=x + 138, column=4).value = f"{unidad[key[x]]} No {num[key[x]]}"
    sheetInforme.cell(row=x + 138, column=16).value = aTerreno[key[x]]
    sheetInforme.cell(row=x + 154, column=11).value = aTechada[key[x]]
    sheetInforme.cell(row=x + 154, column=16).value = aOcupada[key[x]]
    sheetInforme.cell(row=x + 189, column=4).value = f"Es materia de tasación el {unidad[key[x]]} No {num[key[x]]}, ubicado en el nivel {nivel[key[x]]}"
    sheetInforme.cell(row=x + 200, column=13).value = descrip[key[x]]
    sheetInforme.cell(row=x + 287, column=14).value = vut
    sheetInforme.cell(row=x + 324, column=17).value = vrc
    sheetInforme.cell(row=x + 340, column=19).value = vComercialUsd[key[x]]
    y = y + 1

In [33]:
for x in filasInforme:
    for n in range(0, 11-y, 1):
        sheetInforme.row_dimensions[x-n].hidden = True

In [34]:
wbInforme.save('Tasación.xlsx')
wbInforme.close

<bound method Workbook.close of <openpyxl.workbook.workbook.Workbook object at 0x00000206CB6DDC10>>